In [84]:
import numpy as np
from IPython.display import Math, display
import ipywidgets as widgets

In [100]:
decimal_precision = 5


def display_product(A, b):
    line_separator = "\\\\ \n"
    return Math(
        rf"""
\begin{{bmatrix}}
{line_separator
.join(
    [" & "
     .join(
         [str(np.round(A[i, j], decimal_precision)) for j in range(len(A))]
     ) for i in range(len(A))]
)
}
\end{{bmatrix}}
\begin{{bmatrix}}
{line_separator
.join(
    [f"x_{i}" for i in range(1, len(A) + 1)]
)
}
\end{{bmatrix}}
=
\begin{{bmatrix}}
{line_separator
.join(
    [str(np.round(b[i], decimal_precision)) for i in range(len(A))]
)
}
\end{{bmatrix}}
"""
    )


def display_step(A, b, i, j, factor_ij, factor_jj):
    line_separator = "\\\\ \n"
    return Math(
        rf"""
\text{{ ({'I'*(i+1)}) }} - \frac{{ {factor_ij} }}{{ {factor_jj} }} \cdot \text{{ ({'I'*(j+1)}) }} \Rightarrow
    
\begin{{bmatrix}}
{line_separator
.join(
    [" & "
     .join(
         [str(np.round(A[i, j], decimal_precision)) for j in range(len(A))]
     ) for i in range(len(A))]
)
}
\end{{bmatrix}}
\begin{{bmatrix}}
{line_separator
.join(
    [f"x_{i}" for i in range(1, len(A) + 1)]
)
}
\end{{bmatrix}}
=
\begin{{bmatrix}}
{line_separator
.join(
    [str(np.round(b[i], decimal_precision)) for i in range(len(A))]
)
}
\end{{bmatrix}}
"""
    )


def display_solution(x):
    line_separator = "\\\\ \n"
    return Math(
        rf"""
\begin{{bmatrix}}
{line_separator
.join(
    [f"x_{i}" for i in range(1, len(x) + 1)]
)
}
\end{{bmatrix}}
=
\begin{{bmatrix}}
{line_separator
.join(
    [str(np.round(x[i], decimal_precision)) for i in range(len(x))]
)
}
\end{{bmatrix}}
"""
    )


def gauss_algorithm(A, b):
    yield display_product(A, b)

    for i in range(1, len(A)):
        for j in range(0, i):
            factor_ij = A[i, j]
            factor_jj = A[j, j]
            factor = factor_ij / factor_jj
            A[i] -= factor * A[j]
            b[i] -= factor * b[j]
            yield display_step(A, b, i, j, factor_ij, factor_jj)

    if A[len(A) - 1, len(A) - 1] == 0 and b[len(A) - 1] == 0:
        yield Math(r"\text{There are infinitely many solutions.}")
    elif A[len(A) - 1, len(A) - 1] == 0 and b[len(A) - 1] != 0:
        yield Math(r"\text{There is no solution.}")
    else:
        x = [0] * len(A)
        for i in range(len(A) - 1, -1, -1):
            for j in range(len(A) - 1, i, -1):
                b[i] -= A[i, j] * x[j]
            x[i] = b[i] / A[i, i]

        yield display_solution(x)


def create_gui(size):
    coeff_matrix = [[widgets.FloatText() for _ in range(size)] for _ in range(size)]
    res_vector = [widgets.FloatText() for _ in range(size)]
    submit_button = widgets.Button(description="Find solution")
    output = widgets.Output()

    def on_submit_clicked(b):
        with output:
            output.clear_output()
            # Retrieve matrix data
            coeff_matrix_data = np.array(
                [[widget.value for widget in row] for row in coeff_matrix]
            )
            res_vector_data = np.array([widget.value for widget in res_vector])
            for step in gauss_algorithm(coeff_matrix_data, res_vector_data):
                display(step)

    submit_button.on_click(on_submit_clicked)

    display(
        widgets.Label("Coefficient matrix"),
        widgets.VBox(
            [
                *[widgets.HBox(row) for row in coeff_matrix],
            ]
        ),
        widgets.Label("Result vector"),
        widgets.VBox([*[item for item in res_vector]]),
        submit_button,
        output,
    )


size_input = widgets.IntSlider(min=2, max=10, step=1, description="Size:", value=3)

widgets.interact(create_gui, size=size_input)

interactive(children=(IntSlider(value=3, description='Size:', max=10, min=2), Output()), _dom_classes=('widget…